## Update Nearest Neighbors Model to Incorporate New Features
1. Load newly merged data
2. Scale numeric data
3. Apply nearest neigbhors model to scaled dataframe
4. Create nearest/recommendations column
5. Clean
6. Test
7. Load data to database

### 1. Load newly merged data

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neighbors import NearestNeighbors

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

/Users/jisha/.pyenv/versions/3.8.6/lib/python3.8/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
df = pd.read_csv('csv/walk_and_school_score_df.csv')

In [3]:
print(df.shape)
df.head()

(374, 86)


,City,State,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Employed,Unemployment,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Rent,Year,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson,Crime Rate per 1000,Crime Rating,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10,Air Quality Index,Diversity Index,Nearest,lat,lon,Walk Score,Transit Score,Bike Score,Walkability,Public Transportation,Bikeability,Total_Schools,Private,Public district,Public charter,% Private,% Public,% Charter,Above Average or Better,% Performing Above Average or Better
0,Akron,OH,267782,129623,138086,1.9,67.4,23.2,0.2,3.6,0.0,3068406,504125,1732548,258991,19.6,26.2,126247,8.5,85.4,10.3,4.1,0.1,32.3,19.8,25.3,6.6,15.6,83.3,8.5,3.1,1.6,0.6,2.7,21.9,911,2020,197882,1782,27,181,328,1246,6568,1686,4305,577,65,42.20,High,274,203,68,3,0,0,0,107,63,43,0,0,132,0,142,0,Good,0.490241,"126,56,332,355,295",41.044852,-81.520048,46,32,46,Car Dependent,Some Transit,Somewhat Bikeable,208.0,130.0,60.0,18.0,0.62,0.29,0.09,10.0,0.05
1,Albany,GA,93421,43571,49835,2.8,29.2,65.2,0.1,0.9,0.0,1025141,199233,547674,89500,29.6,41.9,35704,16.2,71.9,23.7,4.1,0.1,30.7,21.4,23.1,8.1,16.3,78.1,12.4,1.8,2.3,0.7,4.3,19.1,814,2020,74989,790,12,32,165,581,3452,729,2489,234,17,56.57,High,244,182,60,2,0,0,0,114,63,37,0,0,0,0,244,0,Good,0.488771,"162,372,139,351,249",31.567783,-84.161923,25,0,40,Car Dependent,Minimal,Somewhat Bikeable,109.0,86.0,22.0,1.0,0.79,0.20,0.01,2.0,0.02
2,Albany,NY,147794,70664,77096,7.8,60.3,20.1,0.1,7.6,0.0,1917135,331452,1102265,158632,17.4,17.8,74293,6.6,71.6,24.3,3.9,0.0,42.9,20.5,25.2,4.6,6.6,70.4,7.2,10.1,7.3,1.7,2.9,18.9,1205,2020,97221,736,4,60,189,483,2919,445,2315,159,16,37.59,Medium,274,240,34,0,0,0,0,75,53,39,1,0,169,0,104,0,Good,0.584133,"319,358,336,145,79",42.652580,-73.756633,65,54,56,Somewhat Walkable,Good Transit,Bikeable,151.0,117.0,27.0,7.0,0.77,0.18,0.05,8.0,0.05
3,Albany,OR,61555,30378,31168,11.4,82.7,0.6,0.7,1.9,0.1,542299,75753,268177,35254,14.3,18.6,27721,8.2,78.5,15.9,5.4,0.1,33.3,18.4,22.7,9.9,15.5,79.0,10.7,0.3,2.6,2.0,5.1,20.8,1197,2020,54993,70,1,10,16,43,1467,128,1242,97,17,27.95,Medium,182,161,21,0,0,0,0,70,51,19,0,0,0,0,182,0,Good,0.302632,"29,132,230,383,202",44.627722,-123.094409,34,17,53,Car Dependent,Minimal,Bikeable,43.0,21.0,22.0,0.0,0.49,0.51,0.00,4.0,0.09
4,Albuquerque,NM,632240,309415,322758,50.4,38.5,2.5,4.0,2.4,0.1,7693803,1270581,4099669,581026,18.2,24.8,292907,6.8,74.5,19.6,5.6,0.2,39.8,20.0,24.0,8.3,7.7,80.2,9.1,1.8,1.8,2.5,4.3,21.7,1013,2020,561920,7596,84,486,1699,5327,26059,0,20634,5425,98,59.89,High,183,101,82,0,0,0,0,100,65,50,1,0,127,0,25,30,Good,0.594958,"359,258,386,360,127",35.106766,-106.629181,42,30,62,Car Dependent,Some Transit,Bikeable,543.0,348.0,140.0,55.0,0.64,0.26,0.10,38.0,0.07


### 2. Scale numeric data

In [4]:
numeric = df.select_dtypes(['number'])

In [5]:
numeric = numeric.drop(columns=['lat', 'lon'])

In [6]:
print(numeric.shape)
numeric.head()

(374, 76)


,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Employed,Unemployment,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Rent,Year,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson,Crime Rate per 1000,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10,Diversity Index,Walk Score,Transit Score,Bike Score,Total_Schools,Private,Public district,Public charter,% Private,% Public,% Charter,Above Average or Better,% Performing Above Average or Better
0,267782,129623,138086,1.9,67.4,23.2,0.2,3.6,0.0,3068406,504125,1732548,258991,19.6,26.2,126247,8.5,85.4,10.3,4.1,0.1,32.3,19.8,25.3,6.6,15.6,83.3,8.5,3.1,1.6,0.6,2.7,21.9,911,2020,197882,1782,27,181,328,1246,6568,1686,4305,577,65,42.20,274,203,68,3,0,0,0,107,63,43,0,0,132,0,142,0,0.490241,46,32,46,208.0,130.0,60.0,18.0,0.62,0.29,0.09,10.0,0.05
1,93421,43571,49835,2.8,29.2,65.2,0.1,0.9,0.0,1025141,199233,547674,89500,29.6,41.9,35704,16.2,71.9,23.7,4.1,0.1,30.7,21.4,23.1,8.1,16.3,78.1,12.4,1.8,2.3,0.7,4.3,19.1,814,2020,74989,790,12,32,165,581,3452,729,2489,234,17,56.57,244,182,60,2,0,0,0,114,63,37,0,0,0,0,244,0,0.488771,25,0,40,109.0,86.0,22.0,1.0,0.79,0.20,0.01,2.0,0.02
2,147794,70664,77096,7.8,60.3,20.1,0.1,7.6,0.0,1917135,331452,1102265,158632,17.4,17.8,74293,6.6,71.6,24.3,3.9,0.0,42.9,20.5,25.2,4.6,6.6,70.4,7.2,10.1,7.3,1.7,2.9,18.9,1205,2020,97221,736,4,60,189,483,2919,445,2315,159,16,37.59,274,240,34,0,0,0,0,75,53,39,1,0,169,0,104,0,0.584133,65,54,56,151.0,117.0,27.0,7.0,0.77,0.18,0.05,8.0,0.05
3,61555,30378,31168,11.4,82.7,0.6,0.7,1.9,0.1,542299,75753,268177,35254,14.3,18.6,27721,8.2,78.5,15.9,5.4,0.1,33.3,18.4,22.7,9.9,15.5,79.0,10.7,0.3,2.6,2.0,5.1,20.8,1197,2020,54993,70,1,10,16,43,1467,128,1242,97,17,27.95,182,161,21,0,0,0,0,70,51,19,0,0,0,0,182,0,0.302632,34,17,53,43.0,21.0,22.0,0.0,0.49,0.51,0.00,4.0,0.09
4,632240,309415,322758,50.4,38.5,2.5,4.0,2.4,0.1,7693803,1270581,4099669,581026,18.2,24.8,292907,6.8,74.5,19.6,5.6,0.2,39.8,20.0,24.0,8.3,7.7,80.2,9.1,1.8,1.8,2.5,4.3,21.7,1013,2020,561920,7596,84,486,1699,5327,26059,0,20634,5425,98,59.89,183,101,82,0,0,0,0,100,65,50,1,0,127,0,25,30,0.594958,42,30,62,543.0,348.0,140.0,55.0,0.64,0.26,0.10,38.0,0.07


### Apply Standard Scaler

In [7]:
# Instantiate encoder/scaler
scaler = StandardScaler()

In [8]:
 standard_df = scaler.fit_transform(numeric)

In [9]:
standard_df = pd.DataFrame(standard_df, columns=numeric.columns)

In [10]:
print(standard_df.shape)
standard_df.head()

(374, 76)


,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Employed,Unemployment,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Rent,Year,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson,Crime Rate per 1000,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10,Diversity Index,Walk Score,Transit Score,Bike Score,Total_Schools,Private,Public district,Public charter,% Private,% Public,% Charter,Above Average or Better,% Performing Above Average or Better
0,0.111039,0.101006,0.120488,-0.845252,0.274125,0.791961,-0.296062,-0.158125,-0.318474,0.022786,0.004355,0.051307,0.050765,0.288844,0.463561,0.097748,0.528395,0.989938,-0.775360,-0.760300,-0.429850,-0.397272,0.092803,0.681721,-0.522502,0.676066,0.621552,-0.547011,0.006414,-0.431971,-0.983399,-0.824347,0.070974,-0.635116,0.0,-0.023028,0.103843,0.197378,0.197607,-0.033880,0.156215,0.007353,0.364187,-0.040755,-0.112757,0.254352,0.353014,0.844699,0.451810,0.582986,-0.069547,-0.316806,-0.152499,-0.10463,-0.212366,0.160002,0.330903,-0.573754,-0.393479,-0.022034,-0.212169,1.093323,-0.252284,0.132976,0.332742,0.790400,-0.237926,0.131697,0.074603,0.155840,0.437534,-0.433127,-0.108273,1.401378,-0.067101,-0.276692
1,-0.382026,-0.393704,-0.370566,-0.797049,-1.526254,3.888135,-0.364118,-0.606398,-0.318474,-0.400106,-0.342089,-0.377215,-0.338528,1.880296,2.357478,-0.422326,3.541603,-1.302287,1.568020,-0.760300,-0.429850,-0.584077,0.531110,-0.207781,-0.122661,0.824926,0.077418,1.122174,-0.216352,-0.254017,-0.918659,0.092548,-0.618488,-0.850152,0.0,-0.244987,-0.156511,-0.122279,-0.344022,-0.173368,-0.123317,-0.230165,-0.138903,-0.224982,-0.311526,-0.181972,1.248065,0.284297,0.067272,0.372627,-0.184132,-0.316806,-0.152499,-0.10463,-0.111536,0.160002,-0.259356,-0.573754,-0.393479,-2.105918,-0.212169,2.930118,-0.252284,0.123460,-1.173515,-0.872079,-0.746409,-0.281928,-0.202835,-0.405296,-0.376584,1.214880,-1.042748,-0.621257,-0.491555,-0.827132
2,-0.228268,-0.237947,-0.218878,-0.529256,-0.060501,0.563434,-0.364118,0.505982,-0.318474,-0.215491,-0.191851,-0.176641,-0.179743,-0.061275,-0.549745,-0.200673,-0.215125,-1.353225,1.672948,-0.876126,-1.447342,0.840309,0.284562,0.641289,-1.055623,-1.237844,-0.728320,-1.103406,1.205924,1.017087,-0.271253,-0.709735,-0.667735,0.016644,0.0,-0.204833,-0.170683,-0.292763,-0.242239,-0.152830,-0.164512,-0.270793,-0.288200,-0.242634,-0.354989,-0.191062,0.065875,0.844699,1.129330,-0.311038,-0.413301,-0.316806,-0.152499,-0.10463,-0.673301,-0.349715,-0.062603,1.156763,-0.393479,0.562085,-0.212169,0.409027,-0.252284,0.740884,1.695546,1.933354,0.609545,-0.106451,-0.007368,-0.331462,-0.089248,1.020997,-1.250409,0.390060,-0.173215,-0.276692
3,-0.472138,-0.469550,-0.474435,-0.336445,0.995219,-0.874076,0.044218,-0.440371,-0.114170,-0.500039,-0.482398,-0.478298,-0.463123,-0.554625,-0.453240,-0.468180,0.410997,-0.181644,0.203963,-0.007433,-0.429850,-0.280519,-0.290715,-0.369509,0.357149,0.654800,0.171595,0.394580,-0.473389,-0.177751,-0.077031,0.550995,-0.199886,-0.001091,0.0,-0.281102,-0.345477,-0.356694,-0.423994,-0.300876,-0.349465,-0.381472,-0.454845,-0.351486,-0.390918,-0.181972,-0.534563,-0.873868,-0.317266,-0.652871,-0.413301,-0.316806,-0.152499,-0.10463,-0.745323,-0.451658,-2.030135,-0.573754,-0.393479,-2.105918,-0.212169,1.813635,-0.252284,-1.081702,-0.527976,0.011113,0.355303,-0.557678,-0.612685,-0.405296,-0.424473,-1.693367,2.176001,-0.874087,-0.385442,0.457229
4,1.141670,1.134625,1.148056,1.752340,-1.087942,-0.734011,2.290069,-0.357357,-0.114170,0.980098,0.875269,0.907401,0.790427,0.066041,0.294677,1.055033,-0.13685

### 3. Apply nearest neigbhors model to scaled dataframe

In [11]:
nn = NearestNeighbors(n_neighbors=6, algorithm='kd_tree', n_jobs=8)
nn.fit(standard_df)

NearestNeighbors(algorithm='kd_tree', n_jobs=8, n_neighbors=6)

In [12]:
def nearest(idx):
    return ','.join(map(str, nn.kneighbors([standard_df.iloc[idx]])[1][0][1:].tolist()))

### 4. Create nearest/recommendations column

In [13]:
df['Index'] = df.index

In [14]:
df['Nearest'] = df['Index'].apply(nearest) 

### 5. Clean

In [15]:
# Changing to percent for consistency
change_to_percent = ['Diversity Index', '% Private', '% Public', '% Charter', '% Performing Above Average or Better' ]
df[change_to_percent] = df[change_to_percent]*100

In [16]:
df = df[['City', 'State', 'lat', 'lon', 'TotalPop', 'Men', 'Women', 
        'Hispanic', 'White', 'Black', 'Native', 'Asian', 'Pacific', 'Diversity Index',
        'Income', 'IncomeErr', 'IncomePerCap', 'IncomePerCapErr', 
        'Poverty', 'ChildPoverty',
        'Employed', 'Unemployment', 'PrivateWork', 'PublicWork', 'SelfEmployed', 'FamilyWork', 
        'Professional', 'Service', 'Office', 'Construction', 'Production', 
        'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp', 'WorkAtHome', 'MeanCommute', 
        'Rent', 'Year', 
        'Population','Violent crime', 'Murder and nonnegligent manslaughter', 'Rape',
        'Robbery', 'Aggravated assault', 'Property crime', 'Burglary',
        'Larceny- theft', 'Motor vehicle theft', 'Arson', 'Crime Rate per 1000', 'Crime Rating', 
        'Days with AQI', 'Good Days', 'Moderate Days',
        'Unhealthy for Sensitive Groups Days', 'Unhealthy Days',
        'Very Unhealthy Days', 'Hazardous Days', 'Max AQI',
        '90th Percentile AQI', 'Median AQI', 'Days CO', 'Days NO2',
        'Days Ozone', 'Days SO2', 'Days PM2.5', 'Days PM10', 'Air Quality Index', 
        'Walk Score', 'Transit Score', 'Bike Score', 'Walkability',
        'Public Transportation', 'Bikeability', 
        'Total_Schools', 'Private', 'Public district', 'Public charter', 
        '% Private', '% Public', '% Charter', 'Above Average or Better',
        '% Performing Above Average or Better', 'Nearest']]

In [17]:
df = df.rename(columns = {'% Private': 'Percent Private', 
                          '% Public': 'Percent Public', 
                          '% Charter': 'Percent Charter',
                          '% Performing Above Average or Better': 'Percent Performing Above Average or Better'
                          })
#% signs are escape signals for postgres

In [18]:
print(df.shape)
df.head()

(374, 86)


,City,State,lat,lon,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,Diversity Index,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Employed,Unemployment,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Rent,Year,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson,Crime Rate per 1000,Crime Rating,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10,Air Quality Index,Walk Score,Transit Score,Bike Score,Walkability,Public Transportation,Bikeability,Total_Schools,Private,Public district,Public charter,Percent Private,Percent Public,Percent Charter,Above Average or Better,Percent Performing Above Average or Better,Nearest
0,Akron,OH,41.044852,-81.520048,267782,129623,138086,1.9,67.4,23.2,0.2,3.6,0.0,49.024069,3068406,504125,1732548,258991,19.6,26.2,126247,8.5,85.4,10.3,4.1,0.1,32.3,19.8,25.3,6.6,15.6,83.3,8.5,3.1,1.6,0.6,2.7,21.9,911,2020,197882,1782,27,181,328,1246,6568,1686,4305,577,65,42.20,High,274,203,68,3,0,0,0,107,63,43,0,0,132,0,142,0,Good,46,32,46,Car Dependent,Some Transit,Somewhat Bikeable,208.0,130.0,60.0,18.0,62.0,29.0,9.0,10.0,5.0,"54,123,135,336,89"
1,Albany,GA,31.567783,-84.161923,93421,43571,49835,2.8,29.2,65.2,0.1,0.9,0.0,48.877104,1025141,199233,547674,89500,29.6,41.9,35704,16.2,71.9,23.7,4.1,0.1,30.7,21.4,23.1,8.1,16.3,78.1,12.4,1.8,2.3,0.7,4.3,19.1,814,2020,74989,790,12,32,165,581,3452,729,2489,234,17,56.57,High,244,182,60,2,0,0,0,114,63,37,0,0,0,0,244,0,Good,25,0,40,Car Dependent,Minimal,Somewhat Bikeable,109.0,86.0,22.0,1.0,79.0,20.0,1.0,2.0,2.0,"155,352,333,134,282"
2,Albany,NY,42.652580,-73.756633,147794,70664,77096,7.8,60.3,20.1,0.1,7.6,0.0,58.413267,1917135,331452,1102265,158632,17.4,17.8,74293,6.6,71.6,24.3,3.9,0.0,42.9,20.5,25.2,4.6,6.6,70.4,7.2,10.1,7.3,1.7,2.9,18.9,1205,2020,97221,736,4,60,189,483,2919,445,2315,159,16,37.59,Medium,274,240,34,0,0,0,0,75,53,39,1,0,169,0,104,0,Good,65,54,56,Somewhat Walkable,Good Transit,Bikeable,151.0,117.0,27.0,7.0,77.0,18.0,5.0,8.0,5.0,"304,140,339,328,61"
3,Albany,OR,44.627722,-123.094409,61555,30378,31168,11.4,82.7,0.6,0.7,1.9,0.1,30.263249,542299,75753,268177,35254,14.3,18.6,27721,8.2,78.5,15.9,5.4,0.1,33.3,18.4,22.7,9.9,15.5,79.0,10.7,0.3,2.6,2.0,5.1,20.8,1197,2020,54993,70,1,10,16,43,1467,128,1242,97,17,27.95,Medium,182,161,21,0,0,0,0,70,51,19,0,0,0,0,182,0,Good,34,17,53,Car Dependent,Minimal,Bikeable,43.0,21.0,22.0,0.0,49.0,51.0,0.0,4.0,9.0,"194,360,342,28,164"
4,Albuquerque,NM,35.106766,-106.629181,632240,309415,322758,50.4,38.5,2.5,4.0,2.4,0.1,59.495791,7693803,1270581,4099669,581026,18.2,24.8,292907,6.8,74.5,19.6,5.6,0.2,39.8,20.0,24.0,8.3,7.7,80.2,9.1,1.8,1.8,2.5,4.3,21.7,1013,2020,561920,7596,84,486,1699,5327,26059,0,20634,5425,98,59.89,High,183,101,82,0,0,0,0,100,65,50,1,0,127,0,25,30,Good,42,30,62,Car Dependent,Some Transit,Bikeable,543.0,348.0,140.0,55.0,64.0,26.0,10.0,38.0,7.0,"340,245,75,341,249"


### 6. Test

### Akron, OH
54,123,135,336,89

In [19]:
df[0:1]

,City,State,lat,lon,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,Diversity Index,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Employed,Unemployment,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Rent,Year,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson,Crime Rate per 1000,Crime Rating,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10,Air Quality Index,Walk Score,Transit Score,Bike Score,Walkability,Public Transportation,Bikeability,Total_Schools,Private,Public district,Public charter,Percent Private,Percent Public,Percent Charter,Above Average or Better,Percent Performing Above Average or Better,Nearest
0,Akron,OH,41.044852,-81.520048,267782,129623,138086,1.9,67.4,23.2,0.2,3.6,0.0,49.024069,3068406,504125,1732548,258991,19.6,26.2,126247,8.5,85.4,10.3,4.1,0.1,32.3,19.8,25.3,6.6,15.6,83.3,8.5,3.1,1.6,0.6,2.7,21.9,911,2020,197882,1782,27,181,328,1246,6568,1686,4305,577,65,42.2,High,274,203,68,3,0,0,0,107,63,43,0,0,132,0,142,0,Good,46,32,46,Car Dependent,Some Transit,Somewhat Bikeable,208.0,130.0,60.0,18.0,62.0,29.0,9.0,10.0,5.0,"54,123,135,336,89"


In [20]:
df[54:55]

,City,State,lat,lon,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,Diversity Index,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Employed,Unemployment,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Rent,Year,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson,Crime Rate per 1000,Crime Rating,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10,Air Quality Index,Walk Score,Transit Score,Bike Score,Walkability,Public Transportation,Bikeability,Total_Schools,Private,Public district,Public charter,Percent Private,Percent Public,Percent Charter,Above Average or Better,Percent Performing Above Average or Better,Nearest
54,Canton,OH,40.798946,-81.378448,144372,68904,75423,2.7,78.6,12.9,0.1,0.8,0.0,36.477119,1413199,220336,788406,99988,19.5,28.1,67132,8.6,84.8,10.0,5.0,0.1,29.9,20.9,24.3,8.1,16.5,83.0,9.1,2.2,1.6,0.9,2.8,21.0,795,2020,70139,981,6,135,188,652,3712,962,2429,321,24,66.91,High,274,194,77,3,0,0,0,119,64,43,0,0,156,0,118,0,Good,48,0,46,Car Dependent,Minimal,Somewhat Bikeable,122.0,77.0,38.0,7.0,63.0,31.0,6.0,8.0,7.0,"0,174,354,322,188"


In [21]:
df[123:124]

,City,State,lat,lon,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,Diversity Index,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Employed,Unemployment,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Rent,Year,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson,Crime Rate per 1000,Crime Rating,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10,Air Quality Index,Walk Score,Transit Score,Bike Score,Walkability,Public Transportation,Bikeability,Total_Schools,Private,Public district,Public charter,Percent Private,Percent Public,Percent Charter,Above Average or Better,Percent Performing Above Average or Better,Nearest
123,Fort Wayne,IN,41.093842,-85.139236,310297,150404,159837,7.9,71.4,12.8,0.1,4.0,0.0,46.597938,3895090,609055,1984433,288140,16.0,22.0,148626,6.9,87.5,8.4,3.9,0.1,33.9,16.7,24.5,6.9,17.6,84.4,8.8,0.9,1.1,0.9,3.7,21.0,831,2020,269366,974,26,145,357,446,7437,990,5879,568,49,31.23,Medium,274,190,82,2,0,0,0,120,61,44,0,0,134,0,140,0,Good,30,22,47,Car Dependent,Minimal,Somewhat Bikeable,313.0,238.0,72.0,3.0,76.0,23.0,1.0,11.0,4.0,"0,59,176,315,132"


In [22]:
df[135:136]

,City,State,lat,lon,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,Diversity Index,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Employed,Unemployment,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Rent,Year,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson,Crime Rate per 1000,Crime Rating,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10,Air Quality Index,Walk Score,Transit Score,Bike Score,Walkability,Public Transportation,Bikeability,Total_Schools,Private,Public district,Public charter,Percent Private,Percent Public,Percent Charter,Above Average or Better,Percent Performing Above Average or Better,Nearest
135,Greenville,SC,34.847165,-82.406049,209390,100266,109076,11.1,60.8,23.5,0.1,2.2,0.1,56.230658,2710295,466925,1628704,257964,18.1,25.9,97810,6.7,85.6,9.4,4.7,0.1,37.1,18.2,23.4,7.0,14.0,81.4,9.9,0.7,2.1,1.4,4.2,20.0,1022,2020,69830,402,4,44,82,272,2805,352,2236,217,1,45.93,High,309,259,49,1,0,0,0,116,54,39,0,0,227,0,82,0,Good,43,19,41,Car Dependent,Minimal,Somewhat Bikeable,181.0,123.0,47.0,11.0,68.0,26.0,6.0,18.0,10.0,"257,0,173,146,54"


In [23]:
df[336:337]

,City,State,lat,lon,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,Diversity Index,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Employed,Unemployment,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Rent,Year,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson,Crime Rate per 1000,Crime Rating,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10,Air Quality Index,Walk Score,Transit Score,Bike Score,Walkability,Public Transportation,Bikeability,Total_Schools,Private,Public district,Public charter,Percent Private,Percent Public,Percent Charter,Above Average or Better,Percent Performing Above Average or Better,Nearest
336,Toledo,OH,41.651031,-83.541939,307155,148295,158807,7.9,61.8,24.8,0.2,1.5,0.0,55.010367,3780969,703194,2159177,344566,25.0,32.9,134906,10.8,84.3,11.1,4.3,0.1,29.9,21.3,22.8,6.7,18.9,82.8,8.4,2.2,2.5,1.5,2.3,19.9,848,2020,273505,2604,34,205,655,1710,9470,2362,6314,794,0,44.15,High,274,220,46,8,0,0,0,136,71,40,0,0,183,7,84,0,Good,45,0,50,Car Dependent,Minimal,Bikeable,339.0,237.0,68.0,34.0,70.0,20.0,10.0,7.0,2.0,"0,315,89,54,135"


In [24]:
df[93:94]

,City,State,lat,lon,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,Diversity Index,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Employed,Unemployment,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Rent,Year,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson,Crime Rate per 1000,Crime Rating,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10,Air Quality Index,Walk Score,Transit Score,Bike Score,Walkability,Public Transportation,Bikeability,Total_Schools,Private,Public district,Public charter,Percent Private,Percent Public,Percent Charter,Above Average or Better,Percent Performing Above Average or Better,Nearest
93,Des Moines,IA,41.619549,-93.598022,220844,109609,111205,12.6,67.8,10.3,0.3,5.6,0.1,51.068716,2955194,466643,1553474,215717,17.6,25.0,113536,6.7,85.2,10.4,4.2,0.1,30.8,20.4,25.3,8.8,14.4,80.9,9.7,2.0,2.8,1.3,3.0,18.8,957,2020,218384,1555,14,117,283,1141,8933,2284,5336,1313,53,48.03,High,305,263,40,0,2,0,0,186,54,36,0,6,187,0,112,0,Good,45,30,42,Car Dependent,Some Transit,Somewhat Bikeable,193.0,129.0,64.0,0.0,67.0,33.0,0.0,1.0,1.0,"257,135,0,123,315"


### New York, NY
67,151,195,88,292

In [25]:
df[233:234]

,City,State,lat,lon,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,Diversity Index,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Employed,Unemployment,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Rent,Year,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson,Crime Rate per 1000,Crime Rating,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10,Air Quality Index,Walk Score,Transit Score,Bike Score,Walkability,Public Transportation,Bikeability,Total_Schools,Private,Public district,Public charter,Percent Private,Percent Public,Percent Charter,Above Average or Better,Percent Performing Above Average or Better,Nearest
233,New York,NY,40.73061,-73.935242,1622796,766919,855732,26.0,46.7,12.6,0.1,11.9,0.0,68.427341,24156471,5335466,20233530,3587477,17.3,19.5,888273,6.6,83.7,8.3,7.8,0.2,60.0,13.9,19.9,2.2,3.8,5.9,1.8,59.6,20.2,5.5,6.7,31.8,2263,2020,8379043,47821,319,2770,13396,31336,122299,9846,106931,5522,0,20.3,Low,275,175,92,8,0,0,0,140,80,46,1,26,169,1,78,0,Good,88,84,70,Very Walkable,Excellent Transit,Very Bikeable,1109.0,723.0,321.0,65.0,65.0,29.0,6.0,132.0,12.0,"67,151,195,88,292"


In [26]:
df[67:68]

,City,State,lat,lon,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,Diversity Index,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Employed,Unemployment,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Rent,Year,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson,Crime Rate per 1000,Crime Rating,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10,Air Quality Index,Walk Score,Transit Score,Bike Score,Walkability,Public Transportation,Bikeability,Total_Schools,Private,Public district,Public charter,Percent Private,Percent Public,Percent Charter,Above Average or Better,Percent Performing Above Average or Better,Nearest
67,Chicago,IL,41.885847,-87.618123,2703118,1312735,1390085,28.9,32.8,30.0,0.1,6.1,0.0,71.517326,42663490,8480870,24927384,4007093,20.6,26.6,1304767,10.9,83.4,11.7,4.7,0.1,40.0,19.8,22.2,5.2,12.5,49.2,7.9,28.2,6.6,3.4,4.6,34.5,1417,2020,2707064,25532,492,1761,7983,15296,80742,9578,62083,9081,416,39.26,Medium,274,91,157,21,5,0,0,197,100,56,0,6,103,15,131,19,Moderate,77,65,73,Very Walkable,Good Transit,Very Bikeable,1849.0,1224.0,542.0,83.0,66.0,29.0,4.0,75.0,4.0,"151,195,88,292,258"


In [27]:
df[151:152]

,City,State,lat,lon,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,Diversity Index,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Employed,Unemployment,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Rent,Year,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson,Crime Rate per 1000,Crime Rating,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10,Air Quality Index,Walk Score,Transit Score,Bike Score,Walkability,Public Transportation,Bikeability,Total_Schools,Private,Public district,Public charter,Percent Private,Percent Public,Percent Charter,Above Average or Better,Percent Performing Above Average or Better,Nearest
151,Houston,TX,29.596156,-95.458709,3073190,1532240,1540692,45.2,23.7,21.7,0.1,7.6,0.0,68.666122,33775328,5973385,18282133,2742408,19.6,27.8,1489149,6.7,83.6,9.2,6.9,0.1,33.9,18.6,22.3,12.0,12.9,78.2,11.1,3.3,1.7,2.1,3.5,28.0,1259,2020,2355606,25257,275,1249,9147,14586,101750,17038,71614,13098,485,53.92,High,183,84,89,8,2,0,0,179,90,53,1,6,89,0,86,1,Moderate,48,37,51,Car Dependent,Some Transit,Bikeable,1950.0,1192.0,617.0,141.0,61.0,32.0,7.0,148.0,8.0,"67,195,292,88,258"


In [28]:
df[195:196]

,City,State,lat,lon,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,Diversity Index,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Employed,Unemployment,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Rent,Year,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson,Crime Rate per 1000,Crime Rating,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10,Air Quality Index,Walk Score,Transit Score,Bike Score,Walkability,Public Transportation,Bikeability,Total_Schools,Private,Public district,Public charter,Percent Private,Percent Public,Percent Charter,Above Average or Better,Percent Performing Above Average or Better,Nearest
195,Los Angeles,CA,33.973093,-118.247896,2432264,1199064,1232847,53.4,20.1,12.7,0.1,11.1,0.1,64.599126,32353748,6396440,18459722,3117103,24.3,31.2,1178617,9.1,79.4,9.2,11.2,0.1,34.8,21.7,22.2,7.4,13.6,64.9,8.7,13.0,4.4,2.8,5.9,30.9,2238,2020,4015546,29400,258,2274,9652,17216,95704,13809,66253,15642,1672,31.15,Medium,183,58,87,25,11,2,0,206,140,62,1,17,106,0,57,2,Moderate,68,53,59,Somewhat Walkable,Good Transit,Bikeable,1694.0,1104.0,410.0,180.0,65.0,24.0,11.0,90.0,5.0,"151,67,292,258,88"


In [29]:
df[88:89]

,City,State,lat,lon,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,Diversity Index,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Employed,Unemployment,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Rent,Year,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson,Crime Rate per 1000,Crime Rating,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10,Air Quality Index,Walk Score,Transit Score,Bike Score,Walkability,Public Transportation,Bikeability,Total_Schools,Private,Public district,Public charter,Percent Private,Percent Public,Percent Charter,Above Average or Better,Percent Performing Above Average or Better,Nearest
88,Dallas,TX,32.996848,-96.792113,1339985,663079,676748,41.0,30.9,22.7,0.1,3.4,0.0,68.37335,18989677,3470226,11662331,1880523,21.1,30.1,659617,6.1,85.7,7.7,6.3,0.1,34.7,18.3,23.4,11.8,11.7,76.4,11.2,4.0,1.8,1.8,4.6,26.4,1473,2020,1363295,11764,198,797,4400,6369,45279,9210,25812,10257,144,41.84,High,214,118,85,10,1,0,0,151,90,48,1,5,118,0,90,0,Good,46,40,51,Car Dependent,Some Transit,Bikeable,883.0,519.0,280.0,84.0,59.0,32.0,10.0,76.0,9.0,"292,258,181,294,23"


In [30]:
df[292:293]  

,City,State,lat,lon,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,Diversity Index,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Employed,Unemployment,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Rent,Year,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson,Crime Rate per 1000,Crime Rating,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10,Air Quality Index,Walk Score,Transit Score,Bike Score,Walkability,Public Transportation,Bikeability,Total_Schools,Private,Public district,Public charter,Percent Private,Percent Public,Percent Charter,Above Average or Better,Percent Performing Above Average or Better,Nearest
292,San Antonio,TX,29.468525,-98.526352,1732885,852008,880668,61.4,27.4,6.7,0.1,2.6,0.1,54.27613,18592374,2990756,8569667,1201361,17.1,23.7,797288,6.4,81.1,13.3,5.4,0.1,34.4,19.7,25.9,9.8,10.0,79.5,10.9,2.8,1.3,1.3,3.9,25.0,1198,2020,1559166,11046,105,1630,1965,7346,67422,8172,51469,7781,181,50.33,High,183,97,82,3,1,0,0,158,77,49,1,1,94,0,87,0,Good,35,36,45,Car Dependent,Some Transit,Somewhat Bikeable,1095.0,592.0,395.0,108.0,54.0,36.0,10.0,97.0,9.0,"88,258,181,23,294"


### Save

In [31]:
df.to_csv('csv/final.csv', index=False)

### 7. Save to Database

In [32]:
!pip install sqlalchemy psycopg2-binary

In [34]:
import sqlalchemy

engine = sqlalchemy.create_engine(DATABASE_URL)
connection = engine.connect()

In [35]:
df = pd.read_csv('csv/final.csv')

In [36]:
df.to_sql('data', con=engine, method='multi', if_exists='replace')
df.to_sql('mytable', con=engine, method='multi', if_exists='replace')

### Check

In [37]:
pd.read_sql('data', con=engine)

,index,City,State,lat,lon,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,Diversity Index,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Employed,Unemployment,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Rent,Year,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson,Crime Rate per 1000,Crime Rating,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10,Air Quality Index,Walk Score,Transit Score,Bike Score,Walkability,Public Transportation,Bikeability,Total_Schools,Private,Public district,Public charter,Percent Private,Percent Public,Percent Charter,Above Average or Better,Percent Performing Above Average or Better,Nearest
0,0,Akron,OH,41.044852,-81.520048,267782,129623,138086,1.9,67.4,23.2,0.2,3.6,0.0,49.024069,3068406,504125,1732548,258991,19.6,26.2,126247,8.5,85.4,10.3,4.1,0.1,32.3,19.8,25.3,6.6,15.6,83.3,8.5,3.1,1.6,0.6,2.7,21.9,911,2020,197882,1782,27,181,328,1246,6568,1686,4305,577,65,42.20,High,274,203,68,3,0,0,0,107,63,43,0,0,132,0,142,0,Good,46,32,46,Car Dependent,Some Transit,Somewhat Bikeable,208.0,130.0,60.0,18.0,62.0,29.0,9.0,10.0,5.0,"54,123,135,336,89"
1,1,Albany,GA,31.567783,-84.161923,93421,43571,49835,2.8,29.2,65.2,0.1,0.9,0.0,48.877104,1025141,199233,547674,89500,29.6,41.9,35704,16.2,71.9,23.7,4.1,0.1,30.7,21.4,23.1,8.1,16.3,78.1,12.4,1.8,2.3,0.7,4.3,19.1,814,2020,74989,790,12,32,165,581,3452,729,2489,234,17,56.57,High,244,182,60,2,0,0,0,114,63,37,0,0,0,0,244,0,Good,25,0,40,Car Dependent,Minimal,Somewhat Bikeable,109.0,86.0,22.0,1.0,79.0,20.0,1.0,2.0,2.0,"155,352,333,134,282"
2,2,Albany,NY,42.652580,-73.756633,147794,70664,77096,7.8,60.3,20.1,0.1,7.6,0.0,58.413267,1917135,331452,1102265,158632,17.4,17.8,74293,6.6,71.6,24.3,3.9,0.0,42.9,20.5,25.2,4.6,6.6,70.4,7.2,10.1,7.3,1.7,2.9,18.9,1205,2020,97221,736,4,60,189,483,2919,445,2315,159,16,37.59,Medium,274,240,34,0,0,0,0,75,53,39,1,0,169,0,104,0,Good,65,54,56,Somewhat Walkable,Good Transit,Bikeable,151.0,117.0,27.0,7.0,77.0,18.0,5.0,8.0,5.0,"304,140,339,328,61"
3,3,Albany,OR,44.627722,-123.094409,61555,30378,31168,11.4,82.7,0.6,0.7,1.9,0.1,30.263249,542299,75753,268177,35254,14.3,18.6,27721,8.2,78.5,15.9,5.4,0.1,33.3,18.4,22.7,9.9,15.5,79.0,10.7,0.3,2.6,2.0,5.1,20.8,1197,2020,54993,70,1,10,16,43,1467,128,1242,97,17,27.95,Medium,182,161,21,0,0,0,0,70,51,19,0,0,0,0,182,0,Good,34,17,53,Car Dependent,Minimal,Bikeable,43.0,21.0,22.0,0.0,49.0,51.0,0.0,4.0,9.0,"194,360,342,28,164"
4,4,Albuquerque,NM,35.106766,-106.629181,632240,309415,322758,50.4,38.5,2.5,4.0,2.4,0.1,59.495791,7693803,1270581,4099669,581026,18.2,24.8,292907,6.8,74.5,19.6,5.6,0.2,39.8,20.0,24.0,8.3,7.7,80.2,9.1,1.8,1.8,2.5,4.3,21.7,1013,2020,561920,7596,84,486,1699,5327,26059,0,20634,5425,98,59.89,High,183,101,82,0,0,0,0,100,65,50,1,0,127,0,25,30,Good,42,30,62,Car Dependent,Some Transit,Bikeable,543.0,348.0,140.0,55.0,64.0,26.0,10.0,38.0,7.0,"340,245,75,341,249"
5,5,Alexandria,LA,31.288519,-92.463349,55530,26523,28995,2.4,39.9,52.6,0.3,2.9,0.0,56.270679,550054,116796,328458,60902,22.1,29.7,22425,10.1,75.0,20.5,3.8,0.6,32.7,26.8,22.7,6.3,11.1,80.1,10.4,2.3,1.8,2.7,2.3,18.6,862,2020,46630,732,9,16,132,575,4180,892,3056,232,0,105.34,High,61,57,3,1,0,0,0,103,47,22,0,0,0,0,61,0,Good,33,0,45,Car Dependent,Minimal,Somewhat Bikeable,74.0,47.0,27.0,0.0,64.0,36.0,0.0,6.0,8.0,"254,156,96,268,290"
6,6,Allentown,PA,40.602729,-75.470955,170700,82654,88016,37.6,48.5,8.2,0.1,3.3,0.0,61.558847,1816429,268992,893772,129624,21.0,29.6,75219,10.3,89.0,6.7,4.1,0.1,29.2,19.1,24.0,6.5,20.9,73.7,13.3,3.6,3.8,0.8,4.5,23.4,1230,2020,121855,471,7,64,170,230,2782,521,2018,243,5,26.70,Low,244,206,38,0,0,0,0,93,54,39,0,4